In [21]:
#Importação de pacotes
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import joblib # Utilizado para salvar o modelo preditivo
from sklearn.preprocessing import LabelEncoder #Utilizada para fazer o OneHotEncoding
from sklearn.metrics import mean_squared_error,precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix
from imblearn import under_sampling, over_sampling #Utilizada para fazer o balanceamento de dados
from imblearn.over_sampling import SMOTE #Utilizada para fazer o balanceamento de dados
from sklearn.preprocessing import MinMaxScaler #Utilizada para fazer a padronização dos dados
from sklearn.metrics import r2_score # Utilizado para medir a acuracia do modelo preditivo
import pymssql as sql #conexão SQL

warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [22]:

#criando a conexao com o db
conexao = sql.connect('DESKTOP-NV3ARNG', 'usuario_python', '123456', 'MODELOS_PREDITIVOS')
# criando o df com a consulta na tabela EXTRACAO_DADOS_SISTEMA
df = pd.read_sql_query('select * from EXTRACAO_DADOS_SISTEMA', conexao)
# fechando conexão com db
conexao.close()


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   NUMERO_CONTRATO                 10415 non-null  int64  
 1   DATA_ASSINATURA_CONTRATO        10415 non-null  object 
 2   TIPO_FINANCIAMENTO              10415 non-null  object 
 3   TAXA_AO_ANO                     10415 non-null  float64
 4   PZ_FINANCIAMENTO                10415 non-null  int64  
 5   CIDADE_CLIENTE                  10415 non-null  object 
 6   ESTADO_CLIENTE                  10415 non-null  object 
 7   RENDA_MENSAL_CLIENTE            10415 non-null  float64
 8   QT_PC_ATRASO                    10415 non-null  int64  
 9   QT_DIAS_PRIM_PC_ATRASO          10415 non-null  int64  
 10  QT_TOTAL_PC_PAGAS               10415 non-null  int64  
 11  VL_TOTAL_PC_PAGAS               10411 non-null  float64
 12  QT_PC_PAGA_EM_DIA               

In [24]:
df.isnull().sum()
#Verifica-se valores 5 valores nulos
#4 na var vl_total_pc_pagas e 1 na var idade_assinatura_contrato


NUMERO_CONTRATO                   0
DATA_ASSINATURA_CONTRATO          0
TIPO_FINANCIAMENTO                0
TAXA_AO_ANO                       0
PZ_FINANCIAMENTO                  0
CIDADE_CLIENTE                    0
ESTADO_CLIENTE                    0
RENDA_MENSAL_CLIENTE              0
QT_PC_ATRASO                      0
QT_DIAS_PRIM_PC_ATRASO            0
QT_TOTAL_PC_PAGAS                 0
VL_TOTAL_PC_PAGAS                 4
QT_PC_PAGA_EM_DIA                 0
QT_DIAS_MIN_ATRASO                0
QT_DIAS_MAX_ATRASO                0
QT_DIAS_MEDIA_ATRASO              0
VALOR_FINANCIAMENTO               0
VALOR_PARCELA                     0
IDADE_DATA_ASSINATURA_CONTRATO    1
INADIMPLENTE_COBRANCA             0
dtype: int64

In [25]:
#Verificando se com os dados de outras colunas consigo tratar os valores nulos
nulas = df[df['VL_TOTAL_PC_PAGAS'].isnull() + df['IDADE_DATA_ASSINATURA_CONTRATO'].isnull()]
nulas.head()

,NUMERO_CONTRATO,DATA_ASSINATURA_CONTRATO,TIPO_FINANCIAMENTO,TAXA_AO_ANO,PZ_FINANCIAMENTO,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,VL_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MAX_ATRASO,QT_DIAS_MEDIA_ATRASO,VALOR_FINANCIAMENTO,VALOR_PARCELA,IDADE_DATA_ASSINATURA_CONTRATO,INADIMPLENTE_COBRANCA
5792,132882,2016-08-23,IMOBILIARIO,9.0,100,SAO PAULO,SP,1800.0,0,0,35,23064.46,30,23,87,44,455000.0,4608.50,NaN,NAO
7517,53380,2017-10-24,IMOBILIARIO,22.0,220,VALPARAISO DE GOIAS,GO,5800.0,62,1837,0,NaN,0,0,0,0,200000.0,1109.09,49.0,SIM
7718,72587,2014-08-11,IMOBILIARIO,18.0,240,BRASILIA,DF,7800.0,100,3006,0,NaN,0,0,0,0,350000.0,1720.83,24.0,SIM
8399,74795,2013-05-10,IMOBILIARIO,19.0,240,BRASILIA,DF,7800.0,114,3464,0,NaN,0,0,0,0,550000.0,2727.08,43.0,SIM
10197,79980,2017-02-21,IMOBILIARIO,17.0,72,BRASILIA,DF,7800.0,69,2082,0,NaN,0,0,0,0,280000.0,3983.33,53.0,NAO


In [26]:
#Como são apenas 5 nulos optei por apagar do banco pois não vai afetar no modelo apenas 5 ocorrencias
#tendo em vista o tamanho do dataset
df.dropna(inplace = True)
df.isnull().sum()

NUMERO_CONTRATO                   0
DATA_ASSINATURA_CONTRATO          0
TIPO_FINANCIAMENTO                0
TAXA_AO_ANO                       0
PZ_FINANCIAMENTO                  0
CIDADE_CLIENTE                    0
ESTADO_CLIENTE                    0
RENDA_MENSAL_CLIENTE              0
QT_PC_ATRASO                      0
QT_DIAS_PRIM_PC_ATRASO            0
QT_TOTAL_PC_PAGAS                 0
VL_TOTAL_PC_PAGAS                 0
QT_PC_PAGA_EM_DIA                 0
QT_DIAS_MIN_ATRASO                0
QT_DIAS_MAX_ATRASO                0
QT_DIAS_MEDIA_ATRASO              0
VALOR_FINANCIAMENTO               0
VALOR_PARCELA                     0
IDADE_DATA_ASSINATURA_CONTRATO    0
INADIMPLENTE_COBRANCA             0
dtype: int64